# 高校数学とJulia言語 Day 4

- 城北中学校・高等学校　中学3年・高校1年
- 夏期講習会III 2025/8/24~2025/8/28
- 担当：清水団

## 本日のテーマ：微分・積分の数値計算と最適化問題

今日は数学の花形である微分・積分をJuliaで計算し、現実の問題を解決する最適化技術を学びます！


## 微分・積分と最適化の世界

微分と積分は、変化を扱う数学の根本的な概念です。現代社会では、これらの概念が様々な分野で活用されています。

### 微分の応用例
- **物理学**：速度（位置の微分）、加速度（速度の微分）
- **経済学**：限界費用、限界収益の計算
- **工学**：制御システム、信号処理
- **機械学習**：勾配降下法による学習

### 積分の応用例
- **物理学**：面積、体積、仕事、エネルギーの計算
- **統計学**：確率密度関数の積分
- **工学**：信号の平均値、電気回路の解析
- **経済学**：累積的な効果の計算

### 最適化問題の応用例
- **経営**：利益最大化、コスト最小化
- **工学**：設計の最適化、資源配分
- **物流**：配送ルートの最適化
- **AI**：機械学習モデルのパラメータ最適化


## 必要なパッケージの準備

数値計算と最適化に必要なパッケージをインストールして読み込みましょう。


In [ ]:
# パッケージのインストール（初回のみ実行）
import Pkg
Pkg.add(["Plots", "ForwardDiff", "Optim", "QuadGK", "SymbolicUtils", "Symbolics"])

In [ ]:
# パッケージの読み込み
using Plots
using ForwardDiff      # 自動微分
using Optim            # 最適化
using QuadGK           # 数値積分
using Symbolics        # 記号計算
using LinearAlgebra    # 線形代数

println("パッケージの読み込み完了！")

## 記号計算で微分・積分を理解しよう

まずは記号計算を使って、数式のまま微分・積分を計算してみましょう。


In [ ]:
# 記号変数の定義
@variables x t

println("記号変数 x, t を定義しました")

In [ ]:
# 記号微分の例
println("=== 記号微分の例 ===")

# 基本的な関数の微分
f1 = x^2
f2 = sin(x)
f3 = exp(x)
f4 = x^3 + 2*x^2 - 3*x + 1

println("f(x) = x² の微分：", Symbolics.derivative(f1, x))
println("f(x) = sin(x) の微分：", Symbolics.derivative(f2, x))
println("f(x) = eˣ の微分：", Symbolics.derivative(f3, x))
println("f(x) = x³ + 2x² - 3x + 1 の微分：", Symbolics.derivative(f4, x))

In [ ]:
# 合成関数の微分（連鎖律）
println("=== 合成関数の微分 ===")

f5 = sin(x^2)
f6 = exp(sin(x))
f7 = (x^2 + 1)^3

println("f(x) = sin(x²) の微分：", Symbolics.derivative(f5, x))
println("f(x) = e^(sin(x)) の微分：", Symbolics.derivative(f6, x))
println("f(x) = (x² + 1)³ の微分：", Symbolics.derivative(f7, x))

## 数値微分で実際の値を計算しよう

記号計算で公式を求めたら、次は具体的な値での微分を計算してみましょう。


In [ ]:
# 数値微分の基本
# ForwardDiffパッケージを使用

# 関数を定義
f(x) = x^3 + 2*x^2 - 3*x + 1
g(x) = sin(x^2)
h(x) = exp(x) * cos(x)

# x = 2 での微分値を計算
x_val = 2.0

println("x = ", x_val, " での微分値：")
println("f'(2) = ", ForwardDiff.derivative(f, x_val))
println("g'(2) = ", ForwardDiff.derivative(g, x_val))
println("h'(2) = ", ForwardDiff.derivative(h, x_val))

# 手計算との比較
println("\n手計算での確認：")
println("f(x) = x³ + 2x² - 3x + 1")
println("f'(x) = 3x² + 4x - 3")
println("f'(2) = 3(4) + 4(2) - 3 = 12 + 8 - 3 = 17")
println("数値微分の結果と一致！")

In [ ]:
# 関数とその微分のグラフを描画
x_range = -3:0.1:3
y_values = f.(x_range)
dy_values = ForwardDiff.derivative.(f, x_range)

plot(x_range, y_values, linewidth=3, label="f(x) = x³ + 2x² - 3x + 1", color=:blue)
plot!(x_range, dy_values, linewidth=3, label="f'(x)", color=:red)

# 特定の点での接線を描画
x_point = 1.0
y_point = f(x_point)
slope = ForwardDiff.derivative(f, x_point)

# 接線の方程式: y - y₀ = m(x - x₀)
tangent_x = x_point-1:0.1:x_point+1
tangent_y = slope .* (tangent_x .- x_point) .+ y_point

plot!(tangent_x, tangent_y, linewidth=2, label="x=1での接線", color=:green, linestyle=:dash)
scatter!([x_point], [y_point], markersize=8, color=:green, label="")

title!("関数とその微分")
xlabel!("x")
ylabel!("y")
grid!(true)

## 2階微分で凹凸を調べよう

2階微分は関数の凹凸（曲率）を教えてくれます。最大・最小値を見つけるのに重要です。


In [ ]:
# 2階微分の計算
f_quadratic(x) = x^4 - 4*x^3 + 6*x^2 - 4*x + 1

# 1階微分と2階微分を定義
f_prime(x) = ForwardDiff.derivative(f_quadratic, x)
f_double_prime(x) = ForwardDiff.derivative(f_prime, x)

# 特定の点での値を計算
test_points = [0.5, 1.0, 1.5, 2.0, 2.5]

println("点\t f(x)\t f'(x)\t f''(x)\t 凹凸")
println("="^50)

for x in test_points
    fx = f_quadratic(x)
    fpx = f_prime(x)
    fppx = f_double_prime(x)
    
    concavity = fppx > 0 ? "凸（下凸）" : "凹（上凸）"
    
    println("$(x)\t $(round(fx, digits=2))\t $(round(fpx, digits=2))\t $(round(fppx, digits=2))\t $(concavity)")
end

In [ ]:
# 関数、1階微分、2階微分のグラフ
x_range = 0:0.1:3
y_values = f_quadratic.(x_range)
dy_values = f_prime.(x_range)
d2y_values = f_double_prime.(x_range)

plot(x_range, y_values, linewidth=3, label="f(x)", color=:blue)
plot!(x_range, dy_values, linewidth=3, label="f'(x)", color=:red)
plot!(x_range, d2y_values, linewidth=3, label="f''(x)", color=:green)

# x軸を追加
hline!([0], color=:black, linewidth=1, alpha=0.5, label="")

title!("関数とその1階・2階微分")
xlabel!("x")
ylabel!("y")
grid!(true)
legend!()

println("グラフの読み方：")
println("- f'(x) = 0 の点：関数の極値候補")
println("- f''(x) > 0 の範囲：関数が下凸（凸）")
println("- f''(x) < 0 の範囲：関数が上凸（凹）")
println("- f''(x) = 0 の点：変曲点候補")

## 数値積分で面積を計算しよう

積分は「面積」を求める計算です。Julia の QuadGK パッケージを使って正確に計算できます。


In [ ]:
# 基本的な数値積分
using QuadGK

# 積分する関数を定義
f1(x) = x^2
f2(x) = sin(x)
f3(x) = exp(-x^2)  # ガウス関数
f4(x) = 1/(1 + x^2)  # アークタンジェントの導関数

println("=== 定積分の計算 ===")

# ∫₀¹ x² dx = 1/3
result1, error1 = quadgk(f1, 0, 1)
println("∫₀¹ x² dx = ", result1, " (理論値: 1/3 ≈ ", 1/3, ")")

# ∫₀^π sin(x) dx = 2
result2, error2 = quadgk(f2, 0, π)
println("∫₀^π sin(x) dx = ", result2, " (理論値: 2)")

# ∫₋∞^∞ e^(-x²) dx = √π
result3, error3 = quadgk(f3, -Inf, Inf)
println("∫₋∞^∞ e^(-x²) dx = ", result3, " (理論値: √π ≈ ", sqrt(π), ")")

# ∫₋₁¹ 1/(1+x²) dx = π/2
result4, error4 = quadgk(f4, -1, 1)
println("∫₋₁¹ 1/(1+x²) dx = ", result4, " (理論値: π/2 ≈ ", π/2, ")")

println("\n計算誤差はすべて ", max(error1, error2, error3, error4), " 以下です！")

In [ ]:
# 積分の幾何学的意味を可視化
# sin(x) を 0 から π まで積分

x_range = 0:0.01:π
y_values = sin.(x_range)

# 関数のグラフ
plot(x_range, y_values, linewidth=3, label="y = sin(x)", color=:blue)

# 積分領域を塗りつぶし
fill_x = 0:0.01:π
fill_y = sin.(fill_x)
plot!(fill_x, fill_y, fillrange=0, alpha=0.3, color=:lightblue, label="積分領域")

# 軸を追加
hline!([0], color=:black, linewidth=1, alpha=0.7, label="")
vline!([0], color=:black, linewidth=1, alpha=0.7, label="")

title!("∫₀^π sin(x) dx = 2")
xlabel!("x")
ylabel!("y")
grid!(true)

# 理論値と数値計算値を表示
numerical_result, _ = quadgk(sin, 0, π)
println("数値積分の結果: ", numerical_result)
println("理論値: 2")
println("誤差: ", abs(numerical_result - 2))

## 関数の間の面積を計算しよう

2つの関数に囲まれた面積も積分で求められます。


In [ ]:
# 2つの関数の間の面積
f_upper(x) = x^2 + 1
f_lower(x) = x^2 - 2*x + 1

# 差関数を定義
f_diff(x) = f_upper(x) - f_lower(x)

# 交点を求める（数値的に）
# f_upper(x) = f_lower(x) となる点
# x^2 + 1 = x^2 - 2x + 1
# 2x = 0
# x = 0

# 実際には区間 [-1, 1] で積分してみよう
area, _ = quadgk(f_diff, -1, 1)

println("2つの関数の間の面積: ", area)

# 可視化
x_range = -1.5:0.01:1.5
y_upper = f_upper.(x_range)
y_lower = f_lower.(x_range)

plot(x_range, y_upper, linewidth=3, label="y = x² + 1", color=:blue)
plot!(x_range, y_lower, linewidth=3, label="y = x² - 2x + 1", color=:red)

# 積分区間でのみ塗りつぶし
fill_x = -1:0.01:1
fill_y_upper = f_upper.(fill_x)
fill_y_lower = f_lower.(fill_x)

plot!(fill_x, fill_y_upper, fillrange=fill_y_lower, alpha=0.3, color=:green, label="面積")

title!("2つの関数の間の面積")
xlabel!("x")
ylabel!("y")
grid!(true)
legend!()

# 手計算での確認
println("\n手計算での確認:")
println("f_diff(x) = (x² + 1) - (x² - 2x + 1) = 2x")
println("∫₋₁¹ 2x dx = [x²]₋₁¹ = 1² - (-1)² = 1 - 1 = 0")
println("しかし、|2x| の積分は...")

# 絶対値を考慮した積分
area_abs, _ = quadgk(x -> abs(2*x), -1, 1)
println("∫₋₁¹ |2x| dx = ", area_abs)

## 最適化問題に挑戦しよう

最適化とは「最も良い解」を見つける問題です。微分を使って最大値・最小値を見つけます。


In [ ]:
# 基本的な最適化問題
using Optim

# 最小化する関数を定義
objective_function(x) = x[1]^2 + 2*x[1]*x[2] + 3*x[2]^2 - 4*x[1] - 6*x[2] + 10

# 初期値を設定
initial_guess = [0.0, 0.0]

# 最適化を実行
result = optimize(objective_function, initial_guess, BFGS())

println("最適化結果:")
println("最適解: ", Optim.minimizer(result))
println("最小値: ", Optim.minimum(result))
println("収束したか: ", Optim.converged(result))

# 解析的な解と比較
# ∂f/∂x₁ = 2x₁ + 2x₂ - 4 = 0
# ∂f/∂x₂ = 2x₁ + 6x₂ - 6 = 0
# 解: x₁ = 3, x₂ = -1/3

x1_analytical = 3.0
x2_analytical = -1.0/3.0

println("\n解析解との比較:")
println("解析解: [", x1_analytical, ", ", x2_analytical, "]")
println("最小値（解析）: ", objective_function([x1_analytical, x2_analytical]))

In [ ]:
# 最適化過程の可視化（1変数の場合）
simple_func(x) = (x - 2)^2 + 1

# 最適化を実行
result_1d = optimize(x -> simple_func(x[1]), [0.0], BFGS())
optimal_x = Optim.minimizer(result_1d)[1]
optimal_y = Optim.minimum(result_1d)

# 可視化
x_range = -1:0.1:5
y_values = simple_func.(x_range)

plot(x_range, y_values, linewidth=3, label="f(x) = (x-2)² + 1", color=:blue)
scatter!([optimal_x], [optimal_y], markersize=10, color=:red, label="最適解")

title!("1変数最適化の例")
xlabel!("x")
ylabel!("f(x)")
grid!(true)
legend!()

println("1変数最適化の結果:")
println("最適解 x = ", optimal_x)
println("最小値 f(x) = ", optimal_y)
println("理論値: x = 2, f(x) = 1")

## 実際の問題を解いてみよう

最適化は現実の問題解決に威力を発揮します。いくつかの実例を見てみましょう。


In [ ]:
# 問題1: 箱の体積最大化
# 1辺の長さが20cmの正方形の紙から、4つの角に正方形を切り取って
# 箱を作るとき、体積を最大化する切り取る長さを求める

# 変数: x = 切り取る正方形の1辺の長さ
# 体積 V = x * (20-2x) * (20-2x) = x * (20-2x)²
# 制約: 0 < x < 10

volume_function(x) = x * (20 - 2*x)^2

# 最大化問題を最小化問題に変換（符号を反転）
negative_volume(x) = -volume_function(x[1])

# 制約付き最適化
result_box = optimize(negative_volume, [0.1], [9.9], [5.0], Fminbox(BFGS()))

optimal_cut = Optim.minimizer(result_box)[1]
max_volume = -Optim.minimum(result_box)

println("=== 箱の体積最大化問題 ===")
println("最適な切り取り長さ: ", round(optimal_cut, digits=3), " cm")
println("最大体積: ", round(max_volume, digits=3), " cm³")

# 可視化
x_range = 0.1:0.1:9.9
volumes = volume_function.(x_range)

plot(x_range, volumes, linewidth=3, label="体積 V(x)", color=:blue)
scatter!([optimal_cut], [max_volume], markersize=10, color=:red, label="最適解")

title!("箱の体積最大化")
xlabel!("切り取る長さ (cm)")
ylabel!("体積 (cm³)")
grid!(true)
legend!()

# 解析解との比較
# V'(x) = (20-2x)² + x·2(20-2x)·(-2) = (20-2x)² - 4x(20-2x) = (20-2x)[(20-2x) - 4x] = (20-2x)(20-6x)
# V'(x) = 0 → x = 10 (境界) または x = 20/6 = 10/3
analytical_x = 20/6
analytical_volume = volume_function(analytical_x)

println("\n解析解との比較:")
println("解析解: x = ", round(analytical_x, digits=3), " cm")
println("解析解での体積: ", round(analytical_volume, digits=3), " cm³")

In [ ]:
# 問題2: 利益最大化問題
# ある会社の利益関数が P(x) = -x² + 100x - 1000 (x: 生産量)
# 利益を最大化する生産量を求める

profit_function(x) = -x^2 + 100*x - 1000

# 最大化問題を最小化問題に変換
negative_profit(x) = -profit_function(x[1])

# 最適化実行
result_profit = optimize(negative_profit, [0.0], BFGS())

optimal_production = Optim.minimizer(result_profit)[1]
max_profit = -Optim.minimum(result_profit)

println("=== 利益最大化問題 ===")
println("最適生産量: ", round(optimal_production, digits=2), " 単位")
println("最大利益: ", round(max_profit, digits=2), " 万円")

# 可視化
x_range = 0:1:100
profits = profit_function.(x_range)

plot(x_range, profits, linewidth=3, label="利益 P(x)", color=:green)
scatter!([optimal_production], [max_profit], markersize=10, color=:red, label="最適解")
hline!([0], color=:black, linewidth=1, alpha=0.5, label="損益分岐点")

title!("利益最大化")
xlabel!("生産量")
ylabel!("利益 (万円)")
grid!(true)
legend!()

# 損益分岐点の計算
# P(x) = 0 → -x² + 100x - 1000 = 0
# x² - 100x + 1000 = 0
# x = (100 ± √(10000 - 4000))/2 = (100 ± √6000)/2
discriminant = 100^2 - 4*1000
if discriminant >= 0
    x1 = (100 - sqrt(discriminant))/2
    x2 = (100 + sqrt(discriminant))/2
    println("\n損益分岐点:")
    println("x₁ = ", round(x1, digits=2), " 単位")
    println("x₂ = ", round(x2, digits=2), " 単位")
else
    println("\n損益分岐点なし（常に損失）")
end

## 多変数最適化にチャレンジしよう

現実の問題では複数の変数を同時に最適化する必要があります。


In [ ]:
# 有名なRosenbrock関数の最適化
# f(x,y) = (1-x)² + 100(y-x²)²
# 最小値は (1,1) で 0

rosenbrock(x) = (1 - x[1])^2 + 100*(x[2] - x[1]^2)^2

# 異なる初期値で最適化を試す
initial_points = [[-1.0, 1.0], [0.0, 0.0], [2.0, 2.0]]

println("=== Rosenbrock関数の最適化 ===")
println("理論的最適解: (1, 1), 最小値: 0")
println("")

for (i, init) in enumerate(initial_points)
    result = optimize(rosenbrock, init, BFGS())
    
    println("初期値 ", i, ": ", init)
    println("  最適解: ", round.(Optim.minimizer(result), digits=6))
    println("  最小値: ", round(Optim.minimum(result), digits=10))
    println("  収束: ", Optim.converged(result))
    println("")
end

In [ ]:
# 制約付き最適化問題
# 目的関数: f(x,y) = x² + y² (原点からの距離の二乗を最小化)
# 制約条件: g(x,y) = x + y - 1 = 0 (直線 x + y = 1 上の点)

# ラグランジュ乗数法を用いた解析解
# L(x,y,λ) = x² + y² - λ(x + y - 1)
# ∂L/∂x = 2x - λ = 0 → x = λ/2
# ∂L/∂y = 2y - λ = 0 → y = λ/2
# ∂L/∂λ = -(x + y - 1) = 0 → x + y = 1
# 解: x = y = 1/2, λ = 1

println("=== 制約付き最適化問題 ===")
println("目的関数: f(x,y) = x² + y²")
println("制約条件: x + y = 1")
println("")

# 制約を満たす点での目的関数値を計算
# x + y = 1 → y = 1 - x
# f(x, 1-x) = x² + (1-x)² = x² + 1 - 2x + x² = 2x² - 2x + 1

constrained_objective(x) = 2*x[1]^2 - 2*x[1] + 1

result_constrained = optimize(constrained_objective, [0.0], BFGS())

optimal_x = Optim.minimizer(result_constrained)[1]
optimal_y = 1 - optimal_x
min_value = Optim.minimum(result_constrained)

println("数値解:")
println("  x = ", round(optimal_x, digits=6))
println("  y = ", round(optimal_y, digits=6))
println("  f(x,y) = ", round(min_value, digits=6))
println("")
println("解析解:")
println("  x = 0.5, y = 0.5, f(x,y) = 0.5")

# 可視化
x_range = -1:0.1:2
y_range = -1:0.1:2

# 等高線プロット
z = [i^2 + j^2 for i in x_range, j in y_range]
contour(x_range, y_range, z, levels=20, color=:viridis, alpha=0.7)

# 制約直線
constraint_x = -1:0.1:2
constraint_y = 1 .- constraint_x
plot!(constraint_x, constraint_y, linewidth=3, color=:red, label="制約: x + y = 1")

# 最適点
scatter!([optimal_x], [optimal_y], markersize=10, color=:yellow, label="最適解")

title!("制約付き最適化")
xlabel!("x")
ylabel!("y")
legend!()

## 機械学習との繋がり：最小二乗法

機械学習でも最適化が重要な役割を果たします。線形回帰の最小二乗法を実装してみましょう。


In [ ]:
# 最小二乗法による線形回帰
# データ: (x₁, y₁), (x₂, y₂), ..., (xₙ, yₙ)
# モデル: y = ax + b
# 目的関数: Σ(yᵢ - axᵢ - b)² を最小化

# サンプルデータを生成
using Random
Random.seed!(42)

n = 20
x_data = sort(rand(n) * 10)  # 0から10の間のランダムな点
true_a, true_b = 2.5, 1.0   # 真の係数
noise = randn(n) * 0.5       # ノイズ
y_data = true_a * x_data .+ true_b .+ noise

println("=== 線形回帰：最小二乗法 ===")
println("真の係数: a = ", true_a, ", b = ", true_b)
println("")

# 目的関数を定義（残差の二乗和）
function sum_of_squares(params)
    a, b = params
    return sum((y_data .- a * x_data .- b).^2)
end

# 最適化実行
result_regression = optimize(sum_of_squares, [0.0, 0.0], BFGS())

estimated_a, estimated_b = Optim.minimizer(result_regression)
min_error = Optim.minimum(result_regression)

println("最適化による推定:")
println("  a = ", round(estimated_a, digits=4))
println("  b = ", round(estimated_b, digits=4))
println("  残差二乗和 = ", round(min_error, digits=4))

# 解析解との比較
# 解析解: a = (Σxy - nΣxΣy)/(Σx² - n(Σx)²), b = (Σy - aΣx)/n
sum_x = sum(x_data)
sum_y = sum(y_data)
sum_xy = sum(x_data .* y_data)
sum_x2 = sum(x_data.^2)

analytical_a = (sum_xy - n*sum_x*sum_y/n) / (sum_x2 - n*(sum_x/n)^2)
analytical_b = (sum_y - analytical_a*sum_x) / n

println("\n解析解:")
println("  a = ", round(analytical_a, digits=4))
println("  b = ", round(analytical_b, digits=4))

# 可視化
scatter(x_data, y_data, markersize=6, alpha=0.7, label="データ点", color=:blue)

# 真の直線
x_line = 0:0.1:10
y_true = true_a * x_line .+ true_b
plot!(x_line, y_true, linewidth=3, label="真の直線", color=:green)

# 推定直線
y_estimated = estimated_a * x_line .+ estimated_b
plot!(x_line, y_estimated, linewidth=3, label="推定直線", color=:red, linestyle=:dash)

title!("線形回帰：最小二乗法")
xlabel!("x")
ylabel!("y")
legend!()
grid!(true)

## Day 4 の演習問題

今日学んだ微分・積分・最適化の技術を使って、以下の問題に取り組んでみましょう。

### 問題1: 微分の計算と可視化

関数 $f(x) = x^3 - 6x^2 + 9x + 1$ について：

1. 1階微分 $f'(x)$ と2階微分 $f''(x)$ を数値的に計算してください
2. $f(x)$, $f'(x)$, $f''(x)$ を同じグラフに描画してください（範囲: $x \in [-1, 5]$）
3. 極値（極大値・極小値）と変曲点を見つけてください
4. 関数の増減表を作成してください

### 問題2: 積分の計算

以下の定積分を数値的に計算してください：

1. $\int_0^2 (x^2 + 1) dx$
2. $\int_0^{\pi/2} \sin(x) \cos(x) dx$
3. $\int_1^e \frac{1}{x} dx$

また、それぞれの理論値と比較してください。

### 問題3: 最適化問題

以下の実用的な最適化問題を解いてください：

**缶の表面積最小化問題**

体積が500cm³の円筒形の缶を作るとき、表面積を最小にする半径と高さを求めてください。

- 体積の制約: $V = \pi r^2 h = 500$
- 表面積: $S = 2\pi r^2 + 2\pi r h$
- 制約を使って $h = \frac{500}{\pi r^2}$ と表現し、$S(r)$ を最小化してください

### 問題4: 2変数最適化

関数 $f(x,y) = x^2 + y^2 - 2x - 4y + 5$ の最小値とその位置を求めてください。

1. 数値最適化で解を求める
2. 解析的に偏微分を計算して理論解を求める
3. 等高線プロットで可視化する

### 問題5: 自由課題

身近な問題を最適化問題として定式化し、解いてみてください。例：

- 通学時間の最小化
- 小遣いの使い方の最適化
- 勉強時間の配分最適化

問題設定、数式化、解法、結果の解釈まで含めて取り組んでください。


## 解答欄

以下のセルに解答を記入してください。


### 問題1の解答：微分の計算と可視化

In [ ]:
# 問題1: f(x) = x³ - 6x² + 9x + 1 の微分解析
# ここに解答を書いてください


### 問題2の解答：積分の計算

In [ ]:
# 問題2: 定積分の数値計算
# ここに解答を書いてください


### 問題3の解答：缶の表面積最小化

In [ ]:
# 問題3: 缶の表面積最小化問題
# ここに解答を書いてください


### 問題4の解答：2変数最適化

In [ ]:
# 問題4: 2変数最適化問題
# ここに解答を書いてください


### 問題5の解答：自由課題

In [ ]:
# 問題5: 自由課題
# ここに解答を書いてください


## 発展的な内容：勾配降下法

機械学習で重要な勾配降下法を実装してみましょう。


In [ ]:
# 勾配降下法の実装
function gradient_descent(f, x0, learning_rate=0.01, max_iter=1000, tol=1e-6)
    x = copy(x0)
    history = [copy(x)]
    
    for i in 1:max_iter
        # 勾配を計算
        grad = ForwardDiff.gradient(f, x)
        
        # パラメータを更新
        x_new = x - learning_rate * grad
        
        # 収束判定
        if norm(x_new - x) < tol
            println("収束しました（反復回数: ", i, ")")
            break
        end
        
        x = x_new
        push!(history, copy(x))
    end
    
    return x, history
end

# テスト関数
test_function(x) = (x[1] - 2)^2 + (x[2] - 1)^2

# 勾配降下法を実行
initial_point = [0.0, 0.0]
optimal_point, history = gradient_descent(test_function, initial_point, 0.1)

println("=== 勾配降下法の結果 ===")
println("初期点: ", initial_point)
println("最適解: ", round.(optimal_point, digits=6))
println("最小値: ", round(test_function(optimal_point), digits=6))
println("理論解: [2, 1], 最小値: 0")

# 最適化過程の可視化
x_range = -1:0.1:3
y_range = -1:0.1:2.5
z = [test_function([i, j]) for i in x_range, j in y_range]

contour(x_range, y_range, z, levels=15, color=:viridis, alpha=0.7)

# 勾配降下法の経路を描画
path_x = [point[1] for point in history]
path_y = [point[2] for point in history]
plot!(path_x, path_y, linewidth=3, color=:red, marker=:circle, markersize=4, label="勾配降下法の経路")

# 開始点と終了点を強調
scatter!([initial_point[1]], [initial_point[2]], markersize=10, color=:blue, label="開始点")
scatter!([optimal_point[1]], [optimal_point[2]], markersize=10, color=:red, label="最適解")

title!("勾配降下法による最適化")
xlabel!("x₁")
ylabel!("x₂")
legend!()

## まとめ

今日は微分・積分・最適化という数学の重要な概念をJuliaで実装し、様々な問題に応用しました。

### 学習した内容

#### 微分
- 記号微分による公式の導出
- 数値微分による具体的な値の計算
- 1階微分（傾き）と2階微分（曲率）の意味
- 接線の方程式と関数の増減・凹凸の判定

#### 積分
- 数値積分による面積の計算
- 定積分の幾何学的意味
- 2つの関数の間の面積計算
- 理論値と数値計算の比較

#### 最適化
- 1変数・多変数最適化問題の解法
- 制約付き最適化問題
- 実用的な問題（箱の体積最大化、利益最大化）
- 機械学習との関連（最小二乗法）

### 現代社会での応用

今日学んだ技術は、以下のような分野で実際に使われています：

- **AI・機械学習**：ニューラルネットワークの学習
- **工学**：構造物の最適設計
- **経済学**：資源配分の最適化
- **物理学**：エネルギー最小化原理
- **生物学**：進化のモデル化
- **金融**：リスク最小化ポートフォリオ

### 数学的思考の重要性

微分・積分・最適化を通じて学んだのは、単なる計算技術ではありません：

1. **変化を数量化する思考**：微分による変化率の理解
2. **全体を把握する視点**：積分による累積効果の理解
3. **最適解を求める姿勢**：制約の中でのベストな選択
4. **数値と理論の融合**：コンピュータと数学の協調

### 次回予告

Day 5では、これまでの学習を統合し、実際のプロジェクトに挑戦します。データ分析から最適化まで、総合的な問題解決能力を身につけましょう！

### 提出について

このノートブックに解答を記入し、保存してからGoogle Classroomに提出してください。特に問題5の自由課題では、創造性と数学的思考力を発揮してください！

### 参考資料

- Julia Calculus Documentation: https://docs.julialang.org/
- ForwardDiff.jl: https://github.com/JuliaDiff/ForwardDiff.jl
- Optim.jl: https://github.com/JuliaOpt/Optim.jl
- QuadGK.jl: https://github.com/JuliaMath/QuadGK.jl
- Symbolics.jl: https://github.com/JuliaSymbolics/Symbolics.jl

### 今後の学習へのアドバイス

微分・積分・最適化は数学の基礎であり、様々な分野への扉です。今日の学習を基に：

- 物理学での運動方程式
- 経済学での限界分析
- 統計学での最尤推定
- 機械学習でのバックプロパゲーション

などの発展的な内容にも挑戦してみてください。Julia言語は、これらすべての分野で強力なツールとなるでしょう。

数学とプログラミングの融合により、理論と実践の両方を理解できる力を身につけることができました。この経験を生かし、将来の学習や研究、そして社会での問題解決に活用してください！
